In [1]:
import pandas as pd
!pip install pytreebank

  Created wheel for pytreebank: filename=pytreebank-0.2.7-cp37-none-any.whl size=37070 sha256=ed7ca7b4229813a496173f5f81e68697f99c0e1e4a5dd290e1c96b7521569e49
  Stored in directory: /root/.cache/pip/wheels/e0/b6/91/e9edcdbf464f623628d5c3aa9de28888c726e270b9a29f2368
Successfully built pytreebank


In [2]:
import torch

In [3]:
import pytreebank
import sys
import os


In [4]:
out_path = os.path.join(sys.path[0], 'sst_{}.txt')
dataset = pytreebank.load_sst('./raw_data')

In [5]:
for category in ['train', 'test', 'dev']:
    with open(out_path.format(category), 'w') as outfile:
        for item in dataset[category]:
            outfile.write("__label__{}\t{}\n".format(
                item.to_labeled_lines()[0][0] + 1,
                item.to_labeled_lines()[0][1]
            ))


In [6]:
train_set=dataset['train']


In [7]:
import pandas as pd

In [8]:
df=pd.read_csv('./sst_train.txt',sep='\t',header=None,names=['label','text'])

In [9]:
test_set=pd.read_csv('./sst_test.txt',sep='\t',header=None,names=['label','text'])

In [10]:
val_set=pd.read_csv('./sst_dev.txt',sep='\t',header=None,names=['label','text'])

In [11]:
df['label']=df['label'].str.replace('__label__','')


# New Section

In [12]:
import random
import torch, torchtext
from torchtext import data

In [13]:
# Manual Seed
SEED = 43
torch.manual_seed(SEED)

In [14]:
#Tweet 
Review_text= torchtext.legacy.data.Field(sequential = True, tokenize = 'spacy', batch_first =True, include_lengths=True)
Label = torchtext.legacy.data.LabelField(tokenize ='spacy', is_target=True, batch_first =True, sequential =False)

In [15]:
fields = [('review',Review_text),('label',Label)]

In [16]:
train_example = [torchtext.legacy.data.Example.fromlist([df.text[i],df.label[i]], fields) for i in range(df.shape[0])] 

In [17]:
train_set=torchtext.legacy.data.Dataset(train_example,fields)

In [18]:
test_set['label']=test_set['label'].str.replace('__label__','')
val_set['label']=val_set['label'].str.replace('__label__','')

In [19]:
test_example = [torchtext.legacy.data.Example.fromlist([test_set.text[i],test_set.label[i]], fields) for i in range(test_set.shape[0])] 
val_example = [torchtext.legacy.data.Example.fromlist([val_set.text[i],val_set.label[i]], fields) for i in range(val_set.shape[0])] 

In [20]:
tst_set=torchtext.legacy.data.Dataset(test_example,fields)
vl_set=torchtext.legacy.data.Dataset(val_example,fields)

In [21]:
len(train_set),len(tst_set),len(vl_set)

(8544, 2210, 1101)

Looking at the distribution of different classes in the train_set, test_set and validation_set, I decided use the data augmentation techniques, like back translation, synonym replacement, random insertion, random deletion and random swap to make these datasets balanced.
Once that is done, pickling is done to store these datasets, rather than loading and creating them again and again.


In [22]:
Review_text.build_vocab(train_set)
Label.build_vocab(train_set)

In [23]:
print('Size of input vocab : ', len(Review_text.vocab))
print('Size of label vocab : ', len(Label.vocab))
print('Top 10 words appreared repeatedly :', list(Review_text.vocab.freqs.most_common(10)))
print('Labels : ', Label.vocab.stoi)

Size of input vocab :  17165
Size of label vocab :  5
Top 10 words appreared repeatedly : [('.', 8041), (',', 7131), ('the', 6087), ('and', 4474), ('of', 4446), ('a', 4423), ('to', 3024), ('-', 2739), ("'s", 2544), ('is', 2540)]
Labels :  defaultdict(None, {'4': 0, '2': 1, '3': 2, '5': 3, '1': 4})


In [85]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [86]:
train_iterator = torchtext.legacy.data.BucketIterator(train_set, batch_size = 64, 
                                                            sort_key = lambda x: len(x.review),
                                                            sort_within_batch=True, device = device)

In [87]:
valid_iterator = torchtext.legacy.data.BucketIterator(vl_set, batch_size = 64, 
                                                            sort_key = lambda x: len(x.review),
                                                            sort_within_batch=True, device = device)

In [88]:
import os, pickle
with open('tokenizer.pkl', 'wb') as tokens: 
    #pickle.dump(Tweet.vocab.stoi, tokens)
    pickle.dump(Review_text.vocab.stoi,tokens)

In [130]:
import torch.nn as nn
import torch.nn.functional as F

class classifier(nn.Module):
    
    # Define all the layers used in model
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, dropout):
        
        super(classifier,self).__init__()   
        self.embedding_dim=embedding_dim
        self.hidden_dim=hidden_dim
        self.output_dim=output_dim
        self.n_layers=n_layers       
        self.directions=2
        
        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        # GRU layer
        self.encoder = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           n_layers, 
                           dropout=dropout,
                           batch_first=True,bidirectional=True)
        # try using nn.GRU or nn.RNN here and compare their performances
        # try bidirectional and compare their performances
        
        # Dense layer
        self.fc = nn.Linear(hidden_dim, output_dim)
        #adding another linear layer
        #self.fc2=nn.
        
    def forward(self, text, text_lengths):
        
        # text = [batch size, sent_length]
        embedded = self.embedding(text)
        # embedded = [batch size, sent_len, emb dim]
      
        # packed sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.cpu(), batch_first=True)
        packed_output, (hidden,cell) = self.encoder(packed_embedded)
        #hidden = [batch size, num layers * num directions,hid dim]
        #cell = [batch size, num layers * num directions,hid dim]
    
        # Hidden = [batch size, hid dim * num directions]
        dense_outputs = self.fc(hidden)     
        
        # Final activation function softmax
        output = F.softmax(dense_outputs[0], dim=1)
            
        return output

In [131]:
# Define hyperparameters
size_of_vocab=len(Review_text.vocab)
embedding_dim = 500
num_hidden_nodes = 200
num_output_nodes = 5
num_layers = 2
dropout = 0.2

# Instantiate the model
model = classifier(size_of_vocab, embedding_dim, num_hidden_nodes, num_output_nodes, num_layers, dropout = dropout)

In [132]:
print(model)

#No. of trianable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
    
print(f'The model has {count_parameters(model):,} trainable parameters')

classifier(
  (embedding): Embedding(17165, 500)
  (encoder): LSTM(500, 200, num_layers=2, batch_first=True, dropout=0.2, bidirectional=True)
  (fc): Linear(in_features=200, out_features=5, bias=True)
)
The model has 10,669,905 trainable parameters


In [126]:
device

device(type='cuda')

In [133]:
import torch.optim as optim

# define optimizer and loss
optimizer = optim.Adam(model.parameters(), lr=2e-4)
criterion = nn.CrossEntropyLoss()

# define metric
def binary_accuracy(preds, y):
    #round predictions to the closest integer
    _, predictions = torch.max(preds, 1)
    
    correct = (predictions == y).float() 
    acc = correct.sum() / len(correct)
    return acc

def cat_accuracy(preds,y):
    top_pred=preds.argmax(1,keepdim=True)
    correct=top_pred.eq(y.view_as(top_pred)).sum()
    acc=correct.float()/y.shape[0]
    return acc
    
# push to cuda if available
model = model.to(device)
criterion = criterion.to(device)

In [134]:
b=next(iter(train_iterator))
review,review_length=b.review
    


In [120]:
b=next(iter(valid_iterator))

In [135]:
predictions=model(review,review_length)
predictions.shape

#optimizer.zero_grad()
loss = criterion(predictions, b.label)        
acc = cat_accuracy(predictions, b.label)   
#loss.backward()       
#optimizer.step()  

In [136]:
loss.item()

1.6107863187789917

In [137]:
acc.item()

0.15625

In [74]:
 _, pred = torch.max(predictions, 1)

In [138]:
def train(model, iterator, optimizer, criterion):
    
    # initialize every epoch 
    epoch_loss = 0
    epoch_acc = 0
    
    # set the model in training phase
    model.train()  
    
    for batch in iterator:
        
        # resets the gradients after every batch
        optimizer.zero_grad()   
        
        # retrieve text and no. of words
        #tweet, tweet_lengths = batch.tweet  
        review,review_length = batch.review
        
        # convert to 1D tensor
        #predictions = model(tweet, tweet_lengths).squeeze()  
        predictions=model(review,review_length)
        
        # compute the loss
        loss = criterion(predictions, batch.label)        
        
        # compute the categorical accuracy
        acc = cat_accuracy(predictions, batch.label)   
        
        # backpropage the loss and compute the gradients
        loss.backward()       
        
        # update the weights
        optimizer.step()      
        
        # loss and accuracy
        epoch_loss += loss.item()  
        epoch_acc += acc.item()    
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [139]:
def evaluate(model, iterator, criterion):
    
    # initialize every epoch
    epoch_loss = 0
    epoch_acc = 0

    # deactivating dropout layers
    model.eval()
    
    # deactivates autograd
    with torch.no_grad():
    
        for batch in iterator:
        
            # retrieve text and no. of words
            #tweet, tweet_lengths = batch.tweet
            review,review_length = batch.review
            
            # convert to 1d tensor
            predictions = model(review, review_length).squeeze()
            
            # compute loss and accuracy
            loss = criterion(predictions, batch.label)
            acc = cat_accuracy(predictions, batch.label)
            
            # keep track of loss and accuracy
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [39]:
device

device(type='cuda')

In [140]:
import matplotlib.pyplot as plt
from IPython import display
plt.style.use('seaborn-white')
import numpy as np

In [142]:
N_EPOCHS = 50
best_valid_loss = float('inf')
global plot_iter, plot_loss_train, plot_loss_val
plot_iter = np.zeros((0))
plot_loss_train = np.zeros((0))
plot_loss_val = np.zeros((0))
plot_loss_test=np.zeros((0))
for epoch in range(N_EPOCHS):
     
    # train the model
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    
    # evaluate the model
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    # evaluate the model on test data
    test_loss, test_acc = evaluate(model, test_iterator, criterion)
    # save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')
    
    print(f'\t epoch : {epoch} |\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}% \n')
    print(f'\t Test. Loss: {test_loss:.3f} |  test. Acc: {test_acc*100:.2f}% \n')
    plot_iter = np.append(plot_iter, [epoch])
    plot_loss_train = np.append(plot_loss_train, [train_loss])
    plot_loss_val = np.append(plot_loss_val, [valid_loss])
    plot_loss_test = np.append(plot_loss_test, [test_loss])


	 epoch : 0 |	Train Loss: 0.962 | Train Acc: 94.23%
	 Val. Loss: 1.563 |  Val. Acc: 32.96% 

	 Test. Loss: 1.560 |  test. Acc: 33.26% 

	 epoch : 1 |	Train Loss: 0.962 | Train Acc: 94.22%
	 Val. Loss: 1.564 |  Val. Acc: 32.10% 

	 Test. Loss: 1.565 |  test. Acc: 32.50% 

	 epoch : 2 |	Train Loss: 0.961 | Train Acc: 94.27%
	 Val. Loss: 1.565 |  Val. Acc: 32.45% 

	 Test. Loss: 1.562 |  test. Acc: 32.81% 

	 epoch : 3 |	Train Loss: 0.963 | Train Acc: 94.11%
	 Val. Loss: 1.566 |  Val. Acc: 32.18% 

	 Test. Loss: 1.570 |  test. Acc: 31.43% 

	 epoch : 4 |	Train Loss: 0.974 | Train Acc: 93.18%
	 Val. Loss: 1.576 |  Val. Acc: 31.48% 

	 Test. Loss: 1.580 |  test. Acc: 30.63% 

	 epoch : 5 |	Train Loss: 0.977 | Train Acc: 92.98%
	 Val. Loss: 1.570 |  Val. Acc: 31.74% 

	 Test. Loss: 1.576 |  test. Acc: 31.12% 

	 epoch : 6 |	Train Loss: 0.975 | Train Acc: 93.21%
	 Val. Loss: 1.560 |  Val. Acc: 33.13% 

	 Test. Loss: 1.565 |  test. Acc: 32.46% 

	 epoch : 7 |	Train Loss: 0.965 | Train Acc: 94.

In [ ]:
plt.plot(plot_iter, plot_loss_train, plot_loss_val)
display.clear_output(wait=True)
plt.show()

In [144]:
test_iterator = torchtext.legacy.data.BucketIterator(tst_set, batch_size = 32, 
                                                            sort_key = lambda x: len(x.review),
                                                            sort_within_batch=True, device = device)

In [145]:
test_loss, test_acc = evaluate(model, test_iterator, criterion)
print(f'\t Test. Loss: {test_loss:.3f} |  Test. Acc: {test_acc*100:.2f}% \n')
print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')

	 Test. Loss: 1.574 |  Test. Acc: 31.74% 

	Train Loss: 0.953 | Train Acc: 95.09%


In [45]:
#load weights and tokenizer

path='./saved_weights.pt'
model.load_state_dict(torch.load(path));
model.eval();
tokenizer_file = open('./tokenizer.pkl', 'rb')
tokenizer = pickle.load(tokenizer_file)

#inference 

import spacy
nlp = spacy.load('en')

def classify_tweet(tweet):
    
    categories = {0: "Positive",1:"Negative", 2:"Neutral", 3:"Very Positive", 4:"Very Negative"}
    
    # tokenize the tweet 
    tokenized = [tok.text for tok in nlp.tokenizer(tweet)] 
    # convert to integer sequence using predefined tokenizer dictionary
    indexed = [tokenizer[t] for t in tokenized]        
    # compute no. of words        
    length = [len(indexed)]
    # convert to tensor                                    
    tensor = torch.LongTensor(indexed).to(device)   
    # reshape in form of batch, no. of words           
    tensor = tensor.unsqueeze(1).T  
    # convert to tensor                          
    length_tensor = torch.LongTensor(length)
    # Get the model prediction                  
    prediction = model(tensor, length_tensor)
    print(prediction)
    print(prediction.max())
    print(categories)

    _, pred = torch.max(prediction, 1) 
    print (pred)
    print(pred.item())
    
    return categories[pred.item()]

In [46]:
classify_tweet("never never never buy this product.")

tensor([[2.5751e-05, 9.8939e-01, 9.6328e-03, 4.5416e-04, 4.9328e-04]],
       device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor(0.9894, device='cuda:0', grad_fn=<MaxBackward1>)
{0: 'Positive', 1: 'Negative', 2: 'Neutral', 3: 'Very Positive', 4: 'Very Negative'}
tensor([1], device='cuda:0')
1


'Negative'